### Test the TS Benchmark class for visualization

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import json
from tqdm.auto import tqdm
import copy
# Add the root directory of your project to the Python path
project_root = '/Users/aakankschitnandkeolyar/Desktop/PRISMS'
if project_root not in sys.path:
    sys.path.append(project_root)

from PRISMS.library_analysis.library_analysis_utils import compile_product_scores
from PRISMS.thompson_sampling.legacy.ts_main import *
from PRISMS.library_analysis.visualization import TS_Benchmarks
from PRISMS.thompson_sampling.legacy.baseline import random_baseline

In [42]:
import sys
import os
from tqdm.auto import tqdm
import polars as pl

# Add the PRISMS package to Python path
prisms_path = "/Users/aakankschitnandkeolyar/Desktop/PRISMS"
if prisms_path not in sys.path:
    sys.path.insert(0, prisms_path)
import json
from PRISMS.thompson_sampling import StandardSamplerConfig, run_ts, EnhancedSamplerConfig
from PRISMS.thompson_sampling import RandomBaselineConfig, run_random_baseline
from PRISMS.library_analysis.library_analysis_utils import compile_product_scores
from PRISMS.library_analysis.visualization import TS_Benchmarks
from pydantic import ValidationError

In [15]:
input_json_file = {
"reagent_file_list": [
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/acids.smi",
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/coupled_aa_sub.smi"
    ],
    "reaction_smarts": "[#6:1](=[O:2])[OH].[#7X3;H1,H2;!$(N[!#6]);!$(N[#6]=[O]);!$(N[#6]~[!#6;!#16]):3]>>[#6:1](=[O:2])[#7:3]",
    "num_warmup_trials": 5,
    "num_ts_iterations": 5000,
    "sampler_type": "standard",
    "ts_mode": "minimize",
    "evaluator_class_name": "LookupEvaluator",
    "evaluator_arg": {"ref_filename" : "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores/product_scores.csv",
    "ref_colname" : "Score"},
    "log_filename": "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/notebooks/ts_logs.txt",
    "results_filename": "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/notebooks/ts_results.csv"
}
# input_dict = json.loads(input_json_file)
# Convert evaluator_arg to string (Pydantic expects a string)
# input_json_file["evaluator_arg"] = json.dumps(input_json_file["evaluator_arg"])

In [16]:
# parse_input_dict(input_dict)
config = StandardSamplerConfig.model_validate(input_json_file)

In [17]:
# Generate 10 cycles of standard TS (Greedy Algorithm)
ts_df_list = []
for i in tqdm(range(0,10)):
    ts_df_list.append(run_ts(config, hide_progress=True))

  0%|          | 0/10 [00:00<?, ?it/s]

In [30]:
random_config_data = {
    "evaluator_class_name": "LookupEvaluator",
    "evaluator_arg": {
        "ref_filename": "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores/product_scores.csv",
        "ref_colname": "Score"
    },
    "reaction_smarts": "[#6:1](=[O:2])[OH].[#7X3;H1,H2;!$(N[!#6]);!$(N[#6]=[O]);!$(N[#6]~[!#6;!#16]):3]>>[#6:1](=[O:2])[#7:3]",
    "reagent_file_list": [
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/acids.smi",
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/coupled_aa_sub.smi"
    ],
    "num_trials": 5000,
    "num_to_save": 5000,
    "ascending_output": True,
    "outfile_name": None,  # Optional
    "log_filename": None,
    "results_filename": None
}

In [31]:
random_config = RandomBaselineConfig.model_validate(random_config_data)

In [32]:
# Generate 10 cycles of Random Baseline
random_baseline_df_list = []
for i in tqdm(range(0,10)):
    random_baseline_df_list.append(run_random_baseline(random_config, hide_progress=True))

  0%|          | 0/10 [00:00<?, ?it/s]

In [33]:
standard_boltzmann_config = StandardSamplerConfig.model_validate({
    "reagent_file_list": [
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/acids.smi",
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/coupled_aa_sub.smi"
    ],
    "reaction_smarts": "[#6:1](=[O:2])[OH].[#7X3;H1,H2;!$(N[!#6]);!$(N[#6]=[O]);!$(N[#6]~[!#6;!#16]):3]>>[#6:1](=[O:2])[#7:3]",
    "num_warmup_trials": 5,
    "num_ts_iterations": 5000,
    "sampler_type": "standard",
    "ts_mode": "minimize_boltzmann",  # Boltzmann minimize
    "evaluator_class_name": "LookupEvaluator",
    "evaluator_arg": {
        "ref_filename": "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores/product_scores.csv",
        "ref_colname": "Score"
    },
    "log_filename": None,
    "results_filename": None
})

In [34]:
# Generate 10 runs of Boltzmann Sampling
ts_boltzmann_df_list = []
for i in tqdm(range(0,10)):
    ts_boltzmann_df_list.append(run_ts(standard_boltzmann_config, hide_progress=True))

  0%|          | 0/10 [00:00<?, ?it/s]

In [62]:
enhanced_config = EnhancedSamplerConfig.model_validate({
    "reagent_file_list": [
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/acids.smi",
        "/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/coupled_aa_sub.smi"
    ],
    "reaction_smarts": "[#6:1](=[O:2])[OH].[#7X3;H1,H2;!$(N[!#6]);!$(N[#6]=[O]);!$(N[#6]~[!#6;!#16]):3]>>[#6:1](=[O:2])[#7:3]",
    "num_warmup_trials": 5,
    "num_ts_iterations": 5000,
    "sampler_type": "enhanced",
    "processes": 1,
    "scaling": -1,
    "percent_of_library": 0.1,
    "minimum_no_of_compounds_per_core": 1,
    "stopping_criteria": 5000,
    "evaluator_class_name": "LookupEvaluator",
    "evaluator_arg": {
        "ref_filename": "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores/product_scores.csv",
        "ref_colname": "Score"
    },
    "log_filename": None,
    "results_filename": None
})

In [10]:
# Modify input dict for use with enhanced thompson sampling
input_dict_enhanced_TS = copy.copy(input_dict)
input_dict_enhanced_TS["sampler_type"] = "enhanced"
input_dict_enhanced_TS["processes"] = 1
input_dict_enhanced_TS["percent_of_library"] = 0.1
input_dict_enhanced_TS["scaling"] = -1
input_dict_enhanced_TS["minimum_no_of_compounds_per_core"] = 1
input_dict_enhanced_TS["stopping_criteria"] = 5000

In [63]:
# Generate 10 runs of enhanced thompson sampling
ts_enhanced_df_list = []
for i in tqdm(range(0,10)):
    ts_enhanced_df_list.append(run_ts(enhanced_config, hide_progress=True))

  0%|          | 0/10 [00:00<?, ?it/s]

In [64]:
ts_enhanced_df_list[0]

score,SMILES,Name
f64,str,str
-17.828762,"""CC(C)Oc1ccc(C(=O)N[C@H](CC2CCC…","""CA99_AA28_AA61"""
-10.77541,"""CC(C)Oc1ccc(C(=O)NCC(=O)N[C@@H…","""CA99_AA22_AA21"""
-12.924894,"""CC(C)Oc1ccc(C(=O)N2CC(F)(F)C[C…","""CA99_AA37_AA0"""
-10.529994,"""CC(C)Oc1ccc(C(=O)N(C)CC(=O)N[C…","""CA99_AA41_AA10"""
-10.859008,"""CC(C)Oc1ccc(C(=O)NC2c3ccccc3CC…","""CA99_AA18_AA35"""
…,…,…
-10.367689,"""NC(=O)[C@@H]1C[C@H](F)CN1C(=O)…","""CA42_AA2_AA35"""
-10.038722,"""NC(=O)CN1CN(c2ccccc2)C2(CCN(C(…","""CA15_AA17_AA3"""
-13.006804,"""C#Cc1cccc(C(=O)N2CCC3(CC2)C(=O…","""CA72_AA3_AA46"""


In [26]:
# Get reference data
prod_scores_dir = "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores"
prod_scores_df = compile_product_scores(prod_scores_dir)

In [29]:
# Prep data for visualization
ref_data = prod_scores_df.sort("Scores", descending=False)
ref_data = ref_data.rename({"Product_Code": "Name", "Scores": "score"})
ref_data = ref_data.select(["score", "Name"])
ref_data = ref_data.with_columns(pl.lit("ref").alias("method"))
ref_data = ref_data.with_columns(pl.lit("ref").alias("cycle"))

In [65]:
# Generate Dictionary of TS runs data
ts_datasets_list = [ts_df_list, ts_enhanced_df_list, ts_boltzmann_df_list, random_baseline_df_list]
ts_df_list_dict = {}
for n, method in enumerate(["TS_greedy", "TS_enhanced", "TS_Boltzmann", "Random_Baseline"]):
    ts_df_list_dict[method] = ts_datasets_list[n]

In [66]:
# Create visualization
ts_benchmarks = TS_Benchmarks(no_of_cycles=10, methods_list=["TS_greedy", "TS_enhanced", "TS_Boltzmann", "Random_Baseline"], 
                              TS_runs_data=ts_df_list_dict, 
                              reference_data=ref_data)

🔄 Initializing TS_Benchmarks and generating data...
  TS_greedy: 50000 compounds, score range: -17.829 to -3.760
  TS_enhanced: 307915 compounds, score range: -17.829 to -2.526
  TS_Boltzmann: 50000 compounds, score range: -17.248 to 5.547
  Random_Baseline: 50000 compounds, score range: -16.859 to 10.000

📊 Data Generation Summary:
  - Top-N dataset: (4100, 4) (rows × columns)
  - Complete dataset: (957608, 4) (rows × columns)
  - Cycles analyzed: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
  - Methods: ['Random_Baseline', 'TS_Boltzmann', 'TS_enhanced', 'TS_greedy']
  - Reference compounds included: 100

📊 Barplot Data Summary:
  - Reference baseline: Top 100 compounds from reference data
  - Analysis: How many reference compounds each method found
  - Data shape: (37, 3) (rows × columns)
  - Average compounds found per cycle: 45.1

📈 Generating Line Plot Performance Data...
  Processing method: TS_greedy
  Processing method: TS_enhanced
  Processing method: TS_Boltzmann
  Pro

In [67]:
ts_benchmarks.gen_TS_runs_data()

  TS_greedy: 50000 compounds, score range: -17.829 to -3.760
  TS_enhanced: 307915 compounds, score range: -17.829 to -2.526
  TS_Boltzmann: 50000 compounds, score range: -17.248 to 5.547
  Random_Baseline: 50000 compounds, score range: -16.859 to 10.000

📊 Data Generation Summary:
  - Top-N dataset: (4100, 4) (rows × columns)
  - Complete dataset: (957608, 4) (rows × columns)
  - Cycles analyzed: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
  - Methods: ['Random_Baseline', 'TS_Boltzmann', 'TS_enhanced', 'TS_greedy']
  - Reference compounds included: 100


(shape: (4_100, 4)
 ┌────────────┬─────────────────┬───────────┬───────┐
 │ score      ┆ Name            ┆ method    ┆ cycle │
 │ ---        ┆ ---             ┆ ---       ┆ ---   │
 │ f64        ┆ str             ┆ cat       ┆ str   │
 ╞════════════╪═════════════════╪═══════════╪═══════╡
 │ -17.828762 ┆ CA99_AA28_AA61  ┆ TS_greedy ┆ 1     │
 │ -17.288765 ┆ CA124_AA28_AA61 ┆ TS_greedy ┆ 1     │
 │ -17.248438 ┆ CA95_AA28_AA61  ┆ TS_greedy ┆ 1     │
 │ -17.240562 ┆ CA116_AA28_AA61 ┆ TS_greedy ┆ 1     │
 │ -17.021856 ┆ CA96_AA57_AA59  ┆ TS_greedy ┆ 1     │
 │ …          ┆ …               ┆ …         ┆ …     │
 │ -16.092438 ┆ CA83_AA32_AA61  ┆ ref       ┆ ref   │
 │ -16.090384 ┆ CA110_AA6_AA21  ┆ ref       ┆ ref   │
 │ -16.089331 ┆ CA15_AA22_AA60  ┆ ref       ┆ ref   │
 │ -16.087118 ┆ CA52_AA28_AA61  ┆ ref       ┆ ref   │
 │ -16.079922 ┆ CA72_AA22_AA59  ┆ ref       ┆ ref   │
 └────────────┴─────────────────┴───────────┴───────┘,
 shape: (957_608, 4)
 ┌────────────┬─────────────────┬────────

In [68]:
ts_benchmarks.stripplot_TS_results()

alt.Chart(...)

In [18]:
ts_benchmarks.get_barplot_TS_results_data()

Reference baseline: Top 100 compounds from reference data
Each method shows how many of these 100 compounds it found


cycle,method,found
str,enum,i64
"""10""","""TS_Boltzmann""",7
"""1""","""Random_Baseline""",1
"""4""","""Random_Baseline""",3
"""7""","""Random_Baseline""",2
"""7""","""TS_Boltzmann""",2
…,…,…
"""concat""","""Random_Baseline""",11
"""concat""","""TS_greedy""",78
"""concat""","""TS_enhanced""",93


In [19]:
ts_benchmarks.plot_barplot_TS_results()

alt.LayerChart(...)

In [50]:
ts_benchmarks.gen_line_plot_performance_data()

Starting efficient line plot data generation...
Processing method: TS_greedy
Debug gen_line_plot_performance_data: TS_greedy cycle 1 found 5000 compounds
First 5 compounds: ['CA11_AA46_AA12', 'CA117_AA15_AA61', 'CA96_AA7_AA31', 'CA122_AA21_AA0', 'CA11_AA33_AA12']
Debug: TS_greedy cycle 1 top_n=50: 38/50 = 0.7600
Debug gen_line_plot_performance_data: TS_greedy cycle 2 found 5000 compounds
First 5 compounds: ['CA117_AA15_AA61', 'CA11_AA33_AA12', 'CA117_AA7_AA27', 'CA117_AA61_AA16', 'CA73_AA48_AA61']
Debug: TS_greedy cycle 2 top_n=50: 37/50 = 0.7400
Debug gen_line_plot_performance_data: TS_greedy cycle 3 found 5000 compounds
First 5 compounds: ['CA11_AA46_AA12', 'CA117_AA15_AA61', 'CA11_AA33_AA12', 'CA96_AA44_AA0', 'CA117_AA7_AA27']
Debug: TS_greedy cycle 3 top_n=50: 38/50 = 0.7600
Processing method: TS_enhanced
Processing method: TS_Boltzmann
Processing method: Random_Baseline
Generated line plot data shape: (240, 4)
Top N values tested: [50, 100, 200, 300, 400, 500]
Methods analyzed: ['

cycle,top_n,method,frac_top_n
str,i64,str,f64
"""1""",50,"""TS_greedy""",0.76
"""1""",100,"""TS_greedy""",0.67
"""1""",200,"""TS_greedy""",0.59
"""1""",300,"""TS_greedy""",0.546667
"""1""",400,"""TS_greedy""",0.535
…,…,…,…
"""10""",100,"""Random_Baseline""",0.01
"""10""",200,"""Random_Baseline""",0.01
"""10""",300,"""Random_Baseline""",0.01


In [41]:
ts_benchmarks.line_plot_df.filter((pl.col("top_n")== int(50)) & (pl.col("method")=="TS_greedy"))["frac_top_n"].mean()

0.7545454545454544

In [55]:
ts_benchmarks.plot_line_performance_with_error_bars()

Debug: Methods in data: ['Random_Baseline', 'TS_Boltzmann', 'TS_enhanced', 'TS_greedy']
Debug: Expected methods: ['TS_greedy', 'TS_enhanced', 'TS_Boltzmann', 'Random_Baseline']
Debug: Creating line plot with legend...
Stored in class:
  - grouped_stats: (24, 7) (mean, std, n_cycles, upper, lower)
  - grouped_stats_caps: (24, 9) (includes cap_left, cap_right)
  - unique_top_ns: [50, 100, 200, 300, 400, 500]
  - actual_methods: ['Random_Baseline', 'TS_Boltzmann', 'TS_enhanced', 'TS_greedy']
  - Chart components: line_plot, error_bars, error_caps, final_chart
Debug: Returning final chart with error bars
Debug: You can also test ts_benchmarks.line_only_chart to see if legend works without error bars


alt.LayerChart(...)

In [58]:
ts_benchmarks.get_performance_summary()

{'grouped_stats': shape: (24, 7)
 ┌─────────────────┬───────┬──────────┬──────────┬──────────┬──────────┬──────────┐
 │ method          ┆ top_n ┆ mean     ┆ std      ┆ n_cycles ┆ upper    ┆ lower    │
 │ ---             ┆ ---   ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
 │ str             ┆ i64   ┆ f64      ┆ f64      ┆ u32      ┆ f64      ┆ f64      │
 ╞═════════════════╪═══════╪══════════╪══════════╪══════════╪══════════╪══════════╡
 │ TS_Boltzmann    ┆ 400   ┆ 0.0485   ┆ 0.01912  ┆ 10       ┆ 0.06762  ┆ 0.02938  │
 │ TS_Boltzmann    ┆ 300   ┆ 0.049667 ┆ 0.020334 ┆ 10       ┆ 0.07     ┆ 0.029333 │
 │ TS_enhanced     ┆ 500   ┆ 0.6346   ┆ 0.021314 ┆ 10       ┆ 0.655914 ┆ 0.613286 │
 │ Random_Baseline ┆ 400   ┆ 0.009    ┆ 0.003944 ┆ 10       ┆ 0.012944 ┆ 0.005056 │
 │ TS_greedy       ┆ 300   ┆ 0.552333 ┆ 0.017988 ┆ 10       ┆ 0.570322 ┆ 0.534345 │
 │ …               ┆ …     ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
 │ TS_greedy       ┆ 500   ┆ 0.5314   ┆ 0.0